In [1]:
import argparse
import glob
import logging
import multiprocessing as mp
import os
import time
import cv2

In [2]:
import align_db

In [3]:
print(os.getcwd())

/tf


In [7]:
logger = logging.getLogger(__name__)

In [8]:
align_dlib = align_db.AlignDlib('shape_predictor_68_face_landmarks.dat')

In [9]:
def main(input_dir, output_dir, crop_dim):
    start_time = time.time()
    pool = mp.Pool(processes=mp.cpu_count())

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_dir in os.listdir(input_dir):
        image_output_dir = os.path.join(output_dir, os.path.basename(os.path.basename(image_dir)))
        if not os.path.exists(image_output_dir):
            os.makedirs(image_output_dir)

    image_paths = glob.glob(os.path.join(input_dir, '**/*.jpg'))
    for index, image_path in enumerate(image_paths):
        image_output_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(image_path)))
        output_path = os.path.join(image_output_dir, os.path.basename(image_path))
        pool.apply_async(preprocess_image, (image_path, output_path, crop_dim))

    pool.close()
    pool.join()
    logger.info('Completed in {} seconds'.format(time.time() - start_time))
    
def preprocess_image(input_path, output_path, crop_dim):
    """
    Detect face, align and crop :param input_path. Write output to :param output_path
    :param input_path: Path to input image
    :param output_path: Path to write processed image
    :param crop_dim: dimensions to crop image to
    """
    
    image = _process_image(input_path, crop_dim)
    if image is not None:
        #print('image is there hehe')
        logger.debug('Writing processed file: {}'.format(output_path))
        cv2.imwrite(output_path, image)
    else:
        #print('image is not there at 3AM')
        logger.warning("Skipping filename: {}".format(input_path))
        
def _process_image(filename, crop_dim):
    image = None
    aligned_image = None

    image = _buffer_image(filename)
    if image is not None:
        aligned_image = _align_image(image, crop_dim)
#         print("image:{0}".format(aligned_image))
#         print("anything whatever")
    else:
        raise IOError('Error buffering image: {}'.format(filename))

    return aligned_image

def _buffer_image(filename):
    logger.debug('Reading image: {}'.format(filename))
    image = cv2.imread(filename, )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def _align_image(image, crop_dim):
    bb = align_dlib.getLargestFaceBoundingBox(image)
    ########################################################################################
    #Align is not workin
    ########################################################################################
    aligned = align_dlib.align(crop_dim, image, bb, landmarkIndices=align_db.AlignDlib.INNER_EYES_AND_BOTTOM_LIP)

    if aligned is not None:
        aligned = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
        #print('aligning image')
    else:
        #print('dont have')
    return aligned


In [10]:
logging.basicConfig(level=logging.INFO)
input_dir = 'data/lfw/'
output_dir = 'intermediate/'
crop_dim = 180

main(input_dir, output_dir, crop_dim)
#align_dlib.doghello()

aligning image
aligning image
image:[[[ 85 138 151]
  [ 85 139 152]
  [ 85 139 152]
  ...
  [ 64  97 120]
  [ 59  92 115]
  [ 60  94 116]]

 [[ 83 136 149]
  [ 84 137 150]
  [ 84 139 152]
  ...
  [ 64  96 119]
  [ 58  90 113]
  [ 59  92 115]]

 [[ 80 133 146]
  [ 82 136 149]
  [ 83 138 151]
  ...
  [ 62  94 117]
  [ 57  89 112]
  [ 60  92 115]]

 ...

 [[ 11  57  75]
  [  5  46  64]
  [  0  36  53]
  ...
  [  4  48  68]
  [  2  47  68]
  [  8  53  74]]

 [[  9  57  75]
  [  4  47  64]
  [  0  36  53]
  ...
  [  4  48  68]
  [  2  47  68]
  [  8  53  74]]

 [[  9  58  76]
  [  4  48  65]
  [  0  38  55]
  ...
  [  4  47  67]
  [  3  48  69]
  [  9  54  75]]]image:[[[ 17  24 105]
  [ 19  26 110]
  [ 20  29 115]
  ...
  [ 31  43  46]
  [ 30  44  49]
  [ 30  44  51]]

 [[ 17  23 106]
  [ 19  25 111]
  [ 19  28 116]
  ...
  [ 31  44  48]
  [ 30  44  50]
  [ 31  44  52]]

 [[ 17  22 107]
  [ 18  25 113]
  [ 20  28 119]
  ...
  [ 32  45  49]
  [ 31  45  52]
  [ 32  45  53]]

 ...

 [[ 18  25 

  [140 205 244]]]
anything whatever
image is there hehe
aligning image
image:[[[106 119 141]
  [110 122 143]
  [114 124 144]
  ...
  [103 119 131]
  [ 96 112 125]
  [ 92 107 123]]

 [[103 115 139]
  [107 119 140]
  [110 121 142]
  ...
  [102 118 131]
  [ 95 111 125]
  [ 91 106 123]]

 [[101 113 137]
  [105 117 140]
  [109 120 141]
  ...
  [103 119 132]
  [ 96 111 127]
  [ 92 107 124]]

 ...

 [[195 198 185]
  [187 189 177]
  [183 185 173]
  ...
  [221 227 208]
  [222 228 209]
  [222 228 209]]

 [[198 202 188]
  [189 191 179]
  [185 187 175]
  ...
  [220 226 207]
  [220 226 207]
  [221 227 208]]

 [[202 206 191]
  [192 194 182]
  [188 190 178]
  ...
  [219 225 206]
  [219 225 206]
  [219 225 206]]]aligning image
image:[[[ 93 105 103]
  [102 114 112]
  [115 127 126]
  ...
  [ 31  35  26]
  [ 29  33  24]
  [ 29  32  23]]

 [[ 97 108 106]
  [106 118 116]
  [121 132 130]
  ...
  [ 31  36  27]
  [ 30  34  25]
  [ 30  33  24]]

 [[100 111 109]
  [109 120 118]
  [122 133 132]
  ...
  [ 32  36 

  [ 52  25  18]]]
anything whatever
aligning image
image:[[[ 23  43  31]
  [ 28  49  40]
  [ 34  56  51]
  ...
  [112 129 149]
  [100 116 133]
  [ 96 111 125]]

 [[ 24  44  32]
  [ 29  50  41]
  [ 35  57  52]
  ...
  [107 124 144]
  [ 91 107 124]
  [ 88 104 117]]

 [[ 27  47  34]
  [ 30  51  41]
  [ 36  58  52]
  ...
  [107 124 144]
  [ 91 107 123]
  [ 82  98 111]]

 ...

 [[ 18   3   0]
  [ 19   4   1]
  [ 20   5   2]
  ...
  [ 23   3   0]
  [ 22   3   0]
  [ 20   4   0]]

 [[ 18   3   0]
  [ 19   4   1]
  [ 20   5   2]
  ...
  [ 23   3   0]
  [ 21   2   0]
  [ 19   3   0]]

 [[ 18   3   0]
  [ 19   4   1]
  [ 20   5   2]
  ...
  [ 23   3   0]
  [ 21   2   0]
  [ 19   3   0]]]image is there hehe

anything whatever
image is there hehe
aligning image
image:[[[  4   7  12]
  [  7  10  14]
  [  3   5   7]
  ...
  [ 23  31  49]
  [ 19  27  44]
  [ 16  23  39]]

 [[  4   7  12]
  [  7  10  14]
  [  4   6   8]
  ...
  [ 26  34  53]
  [ 23  31  47]
  [ 19  27  42]]

 [[  2   4  10]
  [  4   7

  [ 53  64  78]]]
aligning image
anything whatever
image:[[[ 24  43  50]
  [ 25  44  52]
  [ 27  46  54]
  ...
  [  0   1   9]
  [  0   1   9]
  [  0   2  10]]

 [[ 24  43  50]
  [ 26  45  53]
  [ 27  46  54]
  ...
  [  1   3  11]
  [  1   3  11]
  [  1   3  11]]

 [[ 24  43  51]
  [ 26  45  53]
  [ 27  46  54]
  ...
  [  3   6  14]
  [  3   6  14]
  [  2   6  14]]

 ...

 [[176 243 241]
  [174 242 242]
  [171 243 241]
  ...
  [ 88 141 165]
  [ 97 150 173]
  [107 160 182]]

 [[180 248 245]
  [180 248 246]
  [178 249 247]
  ...
  [110 163 185]
  [136 189 210]
  [151 205 225]]

 [[183 250 247]
  [182 251 249]
  [181 251 248]
  ...
  [118 170 190]
  [142 194 214]
  [163 216 234]]]image is there hehe

anything whatever
aligning image
image is there hehe
image:[[[105  33  18]
  [ 98  34  21]
  [ 88  34  24]
  ...
  [ 25  41  54]
  [ 23  36  46]
  [ 22  33  39]]

 [[106  33  18]
  [ 98  33  21]
  [ 89  35  26]
  ...
  [ 21  37  51]
  [ 19  32  43]
  [ 19  30  37]]

 [[106  33  17]
  [ 99  34

  [ 26  46  71]]]image:[[[ 47  64 104]
  [ 49  67 107]
  [ 51  69 111]
  ...
  [ 46  49 105]
  [ 52  56 113]
  [ 56  62 122]]

 [[ 46  63 103]
  [ 48  66 106]
  [ 50  69 110]
  ...
  [ 55  59 118]
  [ 67  71 132]
  [ 74  80 144]]

 [[ 46  63 102]
  [ 48  66 106]
  [ 51  69 110]
  ...
  [ 76  79 143]
  [ 85  89 155]
  [ 94  99 168]]

 ...

 [[ 22  28  47]
  [ 27  32  56]
  [ 33  37  65]
  ...
  [ 21  17  17]
  [ 20  16  16]
  [ 18  15  14]]

 [[ 23  28  48]
  [ 28  33  56]
  [ 34  38  66]
  ...
  [ 20  17  17]
  [ 19  16  15]
  [ 17  15  14]]

 [[ 25  29  49]
  [ 28  33  56]
  [ 34  38  66]
  ...
  [ 20  18  16]
  [ 19  17  16]
  [ 18  17  15]]]

anything whatever
image is there hehe
anything whatever
image is there hehe
aligning image
image:[[[110 147 169]
  [113 147 171]
  [116 148 172]
  ...
  [ 56  80 122]
  [ 56  81 123]
  [ 56  81 123]]

 [[109 146 168]
  [111 146 169]
  [114 145 170]
  ...
  [ 56  80 122]
  [ 58  81 123]
  [ 59  83 125]]

 [[110 146 169]
  [112 146 169]
  [115 14

...

anything whatever
image is there hehe
aligning image
image is there hehe
image:[[[129 121 120]
  [101  93  94]
  [ 72  65  67]
  ...
  [ 25  32  51]
  [ 20  27  46]
  [ 18  23  42]]

 [[156 147 147]
  [128 120 121]
  [ 99  91  94]
  ...
  [ 34  43  62]
  [ 27  36  55]
  [ 23  30  48]]

 [[181 172 173]
  [155 148 149]
  [123 116 120]
  ...
  [ 43  54  73]
  [ 36  47  65]
  [ 30  39  58]]

 ...

 [[155 137 130]
  [133 115 108]
  [116  97  90]
  ...
  [ 74 102 136]
  [ 75 103 137]
  [ 76 104 138]]

 [[129 111 104]
  [111  93  86]
  [ 97  78  71]
  ...
  [ 76 104 138]
  [ 77 105 139]
  [ 77 105 139]]

 [[107  89  82]
  [ 96  78  71]
  [ 87  68  61]
  ...
  [ 78 106 140]
  [ 78 106 140]
  [ 79 107 141]]]
anything whatever
image is there hehe
aligning image
image:[[[ 27  50  71]
  [ 36  61  82]
  [ 45  71  92]
  ...
  [ 43  54  67]
  [ 36  46  58]
  [ 30  40  49]]

 [[ 33  57  78]
  [ 43  67  89]
  [ 51  77  99]
  ...
  [ 46  57  71]
  [ 41  52  64]
  [ 37  48  58]]

 [[ 40  63  85]
  [ 50  

  [30 58 89]]]aligning image

aligning image
aligning image
image:[[[ 65 112 146]
  [ 66 114 148]
  [ 66 116 149]
  ...
  [ 72 109 159]
  [ 74 109 159]
  [ 76 109 158]]

 [[ 70 116 150]
  [ 70 117 151]
  [ 71 119 153]
  ...
  [ 73 110 160]
  [ 75 110 159]
  [ 76 109 158]]

 [[ 72 118 152]
  [ 73 120 154]
  [ 74 122 156]
  ...
  [ 75 111 160]
  [ 76 110 160]
  [ 77 111 159]]

 ...

 [[182 157 139]
  [183 157 138]
  [182 156 136]
  ...
  [107  63  46]
  [109  64  48]
  [111  64  50]]

 [[182 159 141]
  [185 160 141]
  [185 160 140]
  ...
  [107  64  47]
  [108  64  48]
  [109  65  49]]

 [[182 162 144]
  [183 162 142]
  [185 162 142]
  ...
  [106  65  46]
  [106  65  47]
  [106  65  48]]]image:[[[  7  36  59]
  [ 12  42  67]
  [ 18  48  75]
  ...
  [ 94 144 164]
  [ 94 144 164]
  [ 94 144 164]]

 [[  8  39  62]
  [ 14  45  70]
  [ 21  51  79]
  ...
  [ 94 144 164]
  [ 94 144 164]
  [ 94 144 164]]

 [[ 12  43  66]
  [ 19  49  75]
  [ 27  57  85]
  ...
  [ 95 144 164]
  [ 94 144 164]
  [ 9

image is there hehe
aligning image
image:[[[ 46 105  93]
  [ 45 106  94]
  [ 45 108  96]
  ...
  [120 141 193]
  [121 142 194]
  [123 143 194]]

 [[ 53 113 101]
  [ 51 114 102]
  [ 50 114 103]
  ...
  [120 141 193]
  [120 141 193]
  [122 142 194]]

 [[ 61 123 111]
  [ 58 122 110]
  [ 56 122 110]
  ...
  [121 140 193]
  [121 141 193]
  [122 142 194]]

 ...

 [[102 101 107]
  [102 100 105]
  [102 100 105]
  ...
  [ 81 103 155]
  [ 79 102 153]
  [ 78 102 152]]

 [[104 102 106]
  [102  99 104]
  [102  98 103]
  ...
  [ 79 103 153]
  [ 78 101 152]
  [ 77 100 151]]

 [[107 104 108]
  [105 101 106]
  [103  99 104]
  ...
  [ 78 101 151]
  [ 77 100 150]
  [ 76  99 149]]]
anything whatever
image is there hehe
aligning image
image:[[[ 3 16 27]
  [ 3 14 24]
  [ 4 13 22]
  ...
  [38 59 89]
  [33 52 79]
  [29 45 69]]

 [[ 3 16 27]
  [ 4 14 24]
  [ 6 14 23]
  ...
  [41 62 93]
  [37 56 84]
  [32 49 75]]

 [[ 3 16 28]
  [ 4 15 25]
  [ 6 16 24]
  ...
  [42 64 96]
  [38 58 87]
  [34 52 79]]

 ...

 [[36 

image:[[[231 229 229]
  [235 234 235]
  [233 233 237]
  ...
  [239 236 244]
  [242 242 251]
  [241 245 254]]

 [[232 231 231]
  [235 234 236]
  [233 233 236]
  ...
  [239 237 245]
  [241 242 251]
  [240 244 253]]

 [[233 231 231]
  [234 234 234]
  [231 231 234]
  ...
  [239 237 245]
  [241 242 250]
  [239 244 253]]

 ...

 [[ 86 115 186]
  [ 79 110 183]
  [ 73 105 180]
  ...
  [204 171 107]
  [202 167 103]
  [201 166 103]]

 [[ 89 116 186]
  [ 82 111 184]
  [ 77 108 182]
  ...
  [207 173 111]
  [204 169 107]
  [203 168 105]]

 [[ 90 116 186]
  [ 84 111 184]
  [ 79 109 183]
  ...
  [207 174 113]
  [205 171 110]
  [204 171 107]]]
aligning image
image:[[[  6  15  28]
  [  9  20  36]
  [ 12  25  43]
  ...
  [144 165 220]
  [143 163 218]
  [147 168 222]]

 [[  4  16  30]
  [  7  21  37]
  [ 10  27  45]
  ...
  [148 168 222]
  [146 166 220]
  [150 170 223]]

 [[  3  19  35]
  [  6  25  42]
  [  9  30  49]
  ...
  [156 175 229]
  [154 173 226]
  [154 173 225]]

 ...

 [[ 21  55 101]
  [ 20  5

  [234 235 243]]]aligning image

image:[[[ 43  57  75]
  [ 45  59  79]
  [ 47  61  82]
  ...
  [ 19  36  49]
  [ 21  37  50]
  [ 21  37  50]]

 [[ 42  56  74]
  [ 44  58  77]
  [ 46  60  81]
  ...
  [ 18  36  49]
  [ 20  36  49]
  [ 20  36  49]]

 [[ 41  56  73]
  [ 42  57  76]
  [ 45  60  80]
  ...
  [ 17  35  48]
  [ 19  35  48]
  [ 19  35  48]]

 ...

 [[ 31  29  29]
  [ 30  30  30]
  [ 28  31  31]
  ...
  [ 63  91 115]
  [ 63  91 114]
  [ 62  90 114]]

 [[ 31  29  29]
  [ 30  29  29]
  [ 28  30  30]
  ...
  [ 64  91 115]
  [ 64  91 114]
  [ 63  91 113]]

 [[ 32  29  29]
  [ 30  29  29]
  [ 28  30  30]
  ...
  [ 65  92 116]
  [ 65  91 114]
  [ 64  91 112]]]anything whatever

image is there hehe
anything whatever
image is there hehe
aligning image
image:[[[132 153 150]
  [130 151 149]
  [129 148 148]
  ...
  [104 116 156]
  [100 112 152]
  [ 97 109 149]]

 [[130 153 150]
  [129 151 148]
  [129 149 148]
  ...
  [105 117 156]
  [102 114 153]
  [ 98 110 150]]

 [[129 153 149]
  [128 151